In [3]:
import sys
sys.path.append('./../../designer/script/')
sys.path.append('./../')
sys.path.append('./../../')

import film as film_mod
import design as design_mod
import spectrum as spec_mod
from optimizer.adam import AdamThicknessOptimizer
from utils.loss import calculate_RMS_f_spec

from analyze_utils.make_design import make_reflection_design

import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy

from analyze_utils.structure import plot_layer_thickness, diff_simple_film
from analyze_utils.result_io import load
from utils.loss import calculate_RMS_f_spec, calculate_RMS
from utils.get_n import get_n_SiO2, get_n_TiO2
from analyze_utils.result_io import save, load

import matplotlib.cm as cmap
import matplotlib.patches as mpatches

In [5]:
def make_rand_new_film_dif_gt(gt):
    layers = 200
    d = np.random.random(layers) * gt / layers * 2 # smaller ot allows for fewer configurations
    film = film_mod.TwoMaterialFilm(1.5, 2.5, 'SiO2', d)
    return film

# Experiment setup (thickness GD)
design = make_reflection_design(make_rand_new_film_dif_gt(0))
target: list[spec_mod.BaseSpectrum] = design.target_specs

rep = 50
gts = np.linspace(200, 5000, 10)

# Run Experiment!
losses_rep = []
best_film_rep = []
for rep_i in range(rep):
    loss_ls = []
    film_ls = []
    print(rep_i / rep)
    for gt in gts:
        film = make_rand_new_film_dif_gt(gt)
        adam_optimizer = AdamThicknessOptimizer(
            film, 
            target, 
            1000, 
            alpha=1, 
            show=False, 
            record=True, 
            batch_size_wl=200
        )
        films, losses = adam_optimizer()
        loss_ls.append(losses)
        film_ls.append(films)
    losses_rep.append(loss_ls)
    best_film_rep.append(film_ls)

losses_rep = np.array(losses_rep)
best_film_rep = np.array(best_film_rep)


0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18
0.2
0.22
0.24
0.26
0.28
0.3
0.32
0.34
0.36
0.38
0.4
0.42
0.44
0.46
0.48
0.5
0.52
0.54
0.56
0.58
0.6
0.62
0.64
0.66
0.68


In [ ]:
working_file = './back_reflector_result_thickness_optimize'
save(working_file, losses_rep, best_film_rep, design)


In [ ]:

def plot_thickness_idx(f):
    d = f.get_d()
    fig, ax = plt.subplots(1, 1)
    wl = np.array([750.])
    s = ax.scatter(range(d.shape[0]), d, marker='o', cmap='coolwarm', c=f.calculate_n_array(wl)[0, :])
    cbar = fig.colorbar(s)
    cbar.set_label('n')

    # target: 800 nm DBR
    wl = 800.
    d_SiO2 = wl / 4 / get_n_SiO2(wl)
    d_TiO2 = wl / 4 / get_n_TiO2(wl)
    
    ax.plot([0, d.shape[0]], [d_SiO2, d_SiO2], c=cmap.get_cmap('coolwarm')(0), alpha=0.4, ls='--')
    ax.plot([0, d.shape[0]], [d_TiO2, d_TiO2], c=cmap.get_cmap('coolwarm')(0.99), alpha=0.4, ls='--')
    ax.set_xlabel('index')
    ax.set_ylabel('thickness')
    return fig, ax

f: film_mod.TwoMaterialFilm = film_rep_ls[40][5][-1]
f.add_spec_param(target[0].INC_ANG, target[0].WLS)
f.remove_negative_thickness_layer()

plot_layer_thickness(f)
_, ax = plot_thickness_idx(f)
ax.set_title(f'loss: {calculate_RMS_f_spec(f, target)}')
